In [1]:
from ultralytics import YOLO
import json
import pandas as pd
from PIL import Image
import os
import traceback
from asyncio import Semaphore
import asyncio


class GenerateAnnotation:
    def __init__(self):
        self.man = ['person']
        self.traffic_signs = ['stop sign','traffic light']
        self.byke = ['motorcycle','bicycle']
        self.cars = ['car','bus','truck']
        self.model = YOLO('myModel.pt').cuda()
        self.image_path = '/home/rakib/Desktop/NAYM/Personal/Personal_Projects/Sir/Accident/dataset/images'
        self.labels_path = '/home/rakib/Desktop/NAYM/Personal/Personal_Projects/Sir/Accident/dataset/labels'
        
    async def ai_detection(self,image):
        try:
            width,height = (Image.open(image)).size
            
            ai_feedback = self.model(image,device=0)
            ai_feedback = json.loads(ai_feedback[0].to_json())
            
            detections = []
            
            for item in ai_feedback:
                if item['name'] in self.man:
                    item['name'] = 'Human'
                    item['class'] = 0
                    detections.append(item)
                    
                elif item['name'] in self.traffic_signs:
                    item['name'] = 'Traffic Sign'
                    item['class'] = 1
                    detections.append(item)
                    
                elif item['name'] in self.byke:
                    item['name'] = 'Bike'
                    item['class'] = 2
                    detections.append(item)
                    
                elif item['name'] in self.cars:
                    item['name'] = 'Car'
                    item['class'] = 3
                    detections.append(item)
            
            return detections,width,height
        except Exception as e:
            traceback.print_exc()
    
    
    async def convert_to_annotation(self,detections,image_width,image_height):
        try:
            annotations = []
            for item in detections:
                class_id = item['class']
                box = item['box']
                xCenter = ((box['x1']+box['x2'])/2)/image_width
                yCenter = ((box['y1']+box['y2'])/2)/image_height
                width = (box['x2']-box['x1'])/image_width
                height = (box['y2']-box['y1'])/image_height
                
                annotations.append(f"{class_id} {xCenter:.6f} {yCenter:.6f} {width:.6f} {height:.6f}")
            return annotations
        except Exception as e:
            traceback.print_exc()
            
            
            
    async def process_image(self,image, semaphore):
        async with semaphore:  # Limit concurrency
            img = f"{self.image_path}/{image}"
            txt_name = image.replace('.jpg', '.txt')
            output_file = f"{self.labels_path}/{txt_name}"

            detections, width, height = await self.ai_detection(img)
            annotation = await self.convert_to_annotation(detections, width, height)

            with open(output_file, 'w') as annotation_file:
                annotation_file.write("\n".join(annotation))

            print(f"Annotation Created for {output_file}")
            
    async def main(self):
        images = os.listdir(self.image_path)
        semaphore = Semaphore(20)  # Allow up to 20 concurrent tasks

        tasks = [self.process_image(image, semaphore) for image in images]
        await asyncio.gather(*tasks)

        data_yaml_content = """
        train: images
        val: images
        test: images

        nc: 4
        names: ['Human', 'Traffic Sign', 'Bike', 'Car']
        """

        # Define the file path
        file_path = f"data.yaml"

        # Write the content to the file
        with open(file_path, "w") as file:
            file.write(data_yaml_content)
        del self.model

In [2]:
AI = GenerateAnnotation()

In [3]:
await AI.main()


image 1/1 /home/rakib/Desktop/NAYM/Personal/Personal_Projects/Sir/Accident/dataset/images/image_1135.jpg: 768x1280 5 persons, 104.5ms
Speed: 3.0ms preprocess, 104.5ms inference, 56.8ms postprocess per image at shape (1, 3, 768, 1280)
Annotation Created for /home/rakib/Desktop/NAYM/Personal/Personal_Projects/Sir/Accident/dataset/labels/image_1135.txt

image 1/1 /home/rakib/Desktop/NAYM/Personal/Personal_Projects/Sir/Accident/dataset/images/image_148.jpg: 768x1280 5 cars, 1 truck, 3 traffic lights, 101.2ms
Speed: 3.0ms preprocess, 101.2ms inference, 0.9ms postprocess per image at shape (1, 3, 768, 1280)
Annotation Created for /home/rakib/Desktop/NAYM/Personal/Personal_Projects/Sir/Accident/dataset/labels/image_148.txt

image 1/1 /home/rakib/Desktop/NAYM/Personal/Personal_Projects/Sir/Accident/dataset/images/image_612.jpg: 768x1280 5 cars, 2 traffic lights, 102.5ms
Speed: 3.8ms preprocess, 102.5ms inference, 0.9ms postprocess per image at shape (1, 3, 768, 1280)
Annotation Created for /h

In [4]:
# import os
# from PIL import Image
# images = os.listdir('images')
# c = 0
# for i in images:
#     im = Image.open(f"images/{i}")
#     c+=1
#     im.save(f"img/image_{c}.jpg","jpeg")